In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import jax.numpy as jnp
from jax import random
import numpyro
import numpyro.distributions as dist
from numpyro.infer import NUTS, MCMC
import arviz as az

rng = np.random.default_rng(42)

# --- Design ---
T = 800                      # days
gamma0, gamma1 = 0.0, 0.02   # true effect of theta on returns
rho = 0.95                   # AR(1) persistence of theta
tau = 0.05                   # innovation sd of theta
sigma_y = 0.01               # return noise sd
sigma_fin = 0.6              # headline measurement sd (per headline)
C_low, C_high = 2, 30        # min/max daily headline counts

# --- Simulate latent theta_t (AR(1)) ---
theta = np.zeros(T)
theta[0] = rng.normal(0, tau/np.sqrt(1-rho**2))
for t in range(1, T):
    theta[t] = rho*theta[t-1] + rng.normal(0, tau)

# --- Simulate headlines and daily mean signal ---
C_t = rng.integers(C_low, C_high+1, size=T)
sbar = np.empty(T)
for t in range(T):
    s_tj = theta[t] + rng.normal(0, sigma_fin, size=C_t[t])    # signals per headline
    sbar[t] = s_tj.mean()                                      # daily average (two-step proxy)

# --- Simulate returns ---
R = gamma0 + gamma1*theta + rng.normal(0, sigma_y, size=T)

def ols(y, x, addc=True):
    X = sm.add_constant(x) if addc else x
    return sm.OLS(y, X).fit()

# 1) Oracle infeasible two-step: R ~ theta (true)
res_oracle = ols(R, theta)

# 2) Feasible two-step: R ~ sbar (proxy with measurement error)
res_feasible = ols(R, sbar)

# 3) Optional: classical ME correction if Var(sbar | theta) known ≈ sigma_fin^2 / C_t
#    Here we use the sample average Var(sbar|theta) as plug-in to de-attenuate slope.
lambda_att = np.var(theta) / np.var(sbar) * np.corrcoef(theta, sbar)[0,1]  # helps interpret
# (for reporting only)

print("--- Oracle (infeasible) ---")
print(res_oracle.params, "\n")
print("--- Feasible two-step ---")
print(res_feasible.params, "\n")

print(f"True gamma1: {gamma1:.4f}")
print(f"Oracle slope (≈ unbiased): {res_oracle.params[1]:.4f}")
print(f"Feasible slope (attenuated): {res_feasible.params[1]:.4f}")
print(f"Corr(theta, sbar): {np.corrcoef(theta, sbar)[0,1]:.3f},  Var(theta)={np.var(theta):.3f}, Var(sbar)={np.var(sbar):.3f}")


--- Oracle (infeasible) ---
[0.00013824 0.01642343] 

--- Feasible two-step ---
[-0.00019924  0.00457487] 

True gamma1: 0.0200
Oracle slope (≈ unbiased): 0.0164
Feasible slope (attenuated): 0.0046
Corr(theta, sbar): 0.578,  Var(theta)=0.019, Var(sbar)=0.053


In [4]:
# === ONE-STEP AR(1) with NumPyro (aggregated measurement) ===
def model_ar1_agg(T, y, sbar, C):
    # AR(1) latent sentiment
    rho = numpyro.sample("rho", dist.Uniform(-0.95, 0.95))
    tau = numpyro.sample("tau", dist.HalfNormal(1.0))
    mu_theta = numpyro.sample("mu_theta", dist.Normal(0.0, 1.0))

    # Stationary init
    sigma0 = tau / jnp.sqrt(1.0 - rho**2 + 1e-8)
    theta0 = numpyro.sample("theta_0", dist.Normal(mu_theta, sigma0))

    # AR(1) recursion
    eps = numpyro.sample("eps", dist.Normal(0.0, 1.0).expand([T-1]).to_event(1))
    def step(prev, e):
        mean_t = mu_theta * (1 - rho) + rho * prev
        return mean_t + tau * e, mean_t + tau * e
    import jax.lax as lax
    _, theta_tail = lax.scan(step, theta0, eps)
    theta = jnp.concatenate([theta0[None], theta_tail])
    numpyro.deterministic("theta", theta)

    # Aggregated daily measurement: s̄_t ~ N(theta_t, sigma_fin / sqrt(C_t))
    sigma_fin = numpyro.sample("sigma_fin", dist.HalfNormal(0.5))
    numpyro.sample("sbar_obs",
                   dist.Normal(theta, sigma_fin / jnp.sqrt(jnp.clip(C, 1.0, 1e9))),
                   obs=sbar)

    # Downstream returns: R_t ~ N(gamma0 + gamma1 * theta_t, sigma_y)
    gamma0 = numpyro.sample("gamma0", dist.Normal(0.0, 0.01))
    gamma1 = numpyro.sample("gamma1", dist.Normal(0.0, 0.01))
    sigma_y = numpyro.sample("sigma_y", dist.HalfNormal(0.01))
    numpyro.sample("y_obs",
                   dist.Normal(gamma0 + gamma1 * theta, sigma_y),
                   obs=y)

# Prepare arrays for JAX
T_jnp   = int(T)
y_jnp   = jnp.asarray(R)
sbar_jnp= jnp.asarray(sbar)
C_jnp   = jnp.asarray(C_t)

# MCMC config (CPU-friendly)
rng_key = random.PRNGKey(0)
nuts    = NUTS(model_ar1_agg, target_accept_prob=0.9)
mcmc    = MCMC(nuts, num_warmup=800, num_samples=1200, num_chains=2, chain_method="sequential", progress_bar=True)
mcmc.run(rng_key, T=T_jnp, y=y_jnp, sbar=sbar_jnp, C=C_jnp)

idata = az.from_numpyro(mcmc)
summ  = az.summary(idata, var_names=["gamma0","gamma1","sigma_y","rho","tau","sigma_fin"], round_to=6)
print(summ)

# Posterior-Mittel von gamma1
g1_mean = float(idata.posterior["gamma1"].mean().values)

# 95%-HDI sauber extrahieren
hdi_ds = az.hdi(idata, var_names=["gamma1"], hdi_prob=0.95)
# hdi_ds ist ein xarray.Dataset mit Variable "gamma1" und Koordinate hdi ∈ {lower, higher}
g1_lo = float(hdi_ds["gamma1"].sel(hdi="lower").values)
g1_hi = float(hdi_ds["gamma1"].sel(hdi="higher").values)

print("\n=== Comparison ===")
print(f"True gamma1:                {gamma1:.6f}")
print(f"Oracle slope (R~theta):     {res_oracle.params[1]:.6f}")
print(f"Feasible two-step (R~sbar): {res_feasible.params[1]:.6f}")
print(f"One-step AR(1) (posterior): mean={g1_mean:.6f}, 95% HDI=({g1_lo:.6f}, {g1_hi:.6f})")


sample: 100%|██████████| 2000/2000 [00:18<00:00, 108.88it/s, 63 steps of size 7.81e-02. acc. prob=0.93]


               mean        sd    hdi_3%   hdi_97%  mcse_mean   mcse_sd  \
gamma0     0.000139  0.000378 -0.000573  0.000838   0.000008  0.000007   
gamma1     0.014273  0.002831  0.009281  0.019805   0.000050  0.000057   
sigma_y    0.010078  0.000259  0.009601  0.010573   0.000003  0.000006   
rho        0.913958  0.021324  0.877349  0.949554   0.000727  0.000489   
tau        0.057469  0.006857  0.044786  0.069491   0.000273  0.000130   
sigma_fin  0.579731  0.018849  0.543457  0.615113   0.000528  0.000308   

              ess_bulk     ess_tail     r_hat  
gamma0     2421.724097  1907.216980  1.000919  
gamma1     3189.070525  1838.719046  1.001102  
sigma_y    5523.741608  1603.398761  0.999464  
rho         867.370436  1360.338867  1.000164  
tau         635.689800  1299.015503  1.002603  
sigma_fin  1267.241589  2132.918017  1.000257  

=== Comparison ===
True gamma1:                0.020000
Oracle slope (R~theta):     0.016423
Feasible two-step (R~sbar): 0.004575
One-step AR(1)